<a href="https://colab.research.google.com/github/GauthamPrabhuM/ACMWinterSchool/blob/main/WSOO_Gradient_Scaling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

$\Large\textbf{Scaling gradients.}$



When we tried to solve certain problems of the form $\min_{\mathbf{x} \in {\mathbb{R}}^n} f(\mathbf{x})$ using gradient descent algorithm, we noticed that the algorithm needed a large number of iterations to find the minimizer (e.g. Rosenbrock's function). We will discuss some remedy measures for this issue.  

Consider the problem $\min_{\mathbf{x}} f(\mathbf{x}) = 1500x_1^2 + 4x_1 x_2 +  x_2^2$. 

Note that the function $f(\mathbf{x})$ is twice continuously differentiable. First let us investigate the Hessian $\nabla^2 f(\mathbf{x})$ of the function, given by:
\begin{align}
\nabla^2 f(\mathbf{x}) = \begin{bmatrix}
                          -4000(x_2-3x_1^2)+2 & -4000 x_1 \\
                          -4000 x_1 & 2000
                          \end{bmatrix}
\end{align}

We shall find the condition number of the Hessian given by $\kappa\left (\nabla^2 f(\mathbf{x}) \right ) = \frac{\lambda_{\max} \left (\nabla^2 f(\mathbf{x}) \right )}{\lambda_{\min} \left (\nabla^2 f(\mathbf{x}) \right )}$, where $\lambda_{\max}(A)$ denotes the maximum eigen value of matrix $A$ and $\lambda_{\min}(A)$ denotes the minimum eigen value of $A$.  



In [ ]:
#numpy package will be used for most of our lab exercises. Please have a look at https://numpy.org/doc/1.19/ for numpy documentation
#we will first import the numpy package and name it as np
import numpy as np 
#Henceforth, we can lazily use np to denote the much longer numpy !! 
import scipy as sp
from scipy.linalg import sqrtm

In [ ]:
def evalh(x):
  assert type(x) is np.ndarray
  assert len(x) == 2
  hess = np.array([[1500., 2.],[2., 1.0]])
  hess_sqrt = sqrtm(hess)
  print(hess_sqrt)
  return hess

In [ ]:
def find_condition_number_point(x):
  assert type(x) is np.ndarray
  assert len(x) == 2
  
  eig_vals = np.linalg.eigvalsh(evalh(x))
  print(eig_vals)
  cond_number = eig_vals[-1]/eig_vals[0]
  print(cond_number)

  cond_check = np.linalg.cond(evalh(x))
  print(cond_check)
  return cond_check
  

def find_condition_number_matrix(A):
  assert type(A) is np.ndarray 
  assert A.shape[0] == A.shape[1]

  #print(A)
  eig_vals = np.linalg.eigvalsh(A)
  #print(eig_vals)
  cond_number = eig_vals[-1]/eig_vals[0]
  #print(cond_number)

  cond_check = np.linalg.cond(A)
  print(cond_check)
  return cond_check


In [ ]:
#A = np.array([[32002, -8000], [-8000, 2000]])
A = np.array([[1500., 2.], [2., 1.]])
find_condition_number_matrix(A)

1504.016046343423


1504.016046343423

In [ ]:
x = np.array([5.,5.])
find_condition_number_point(x)

[[38.72980074  0.05034165]
 [ 0.05034165  0.99873206]]
[9.97331559e-01 1.50000267e+03]
1504.016046343423
[[38.72980074  0.05034165]
 [ 0.05034165  0.99873206]]
1504.016046343423


1504.016046343423

#Is the problem well-conditioned? 

The condition number of the Hessian plays a major role in the progress of the iterates of gradient descent towards the optimal solution point. 

Typically a large value of the condition number indicates that the problem is $\textbf{ill-conditioned}$ and hence leads to slow progress of the iterates towards the optimal solution point. 

Now we shall discuss a method which would help in better $\textbf{conditioning}$ of the problem and hence would help in speeding up the progress of the iterates towards the optimal solution point. 



#Induced Conditioning

Let us first illustrate an equivalent transformation of the problem $\min_{\mathbf{x} \in {\mathbb{R}}^n} f(\mathbf{x})$. 

Consider the transformation $\mathbf{x}=\mathbf{My}$ where $\mathbf{M}\in {\mathbb{R}}^{n \times n}$ is an invertible matrix and $\mathbf{y} \in {\mathbb{R}}^n$.

Now consider the equivalent problem $\min_{\mathbf{y} \in {\mathbb{R}}^n} g(\mathbf{y}) \equiv \min_{\mathbf{y} \in {\mathbb{R}}^n} f(\mathbf{My})$. 

Note that the gradient $\nabla g(\mathbf{y}) = \mathbf{M}^\top \nabla f(\mathbf{My})$ and the Hessian is $\nabla^2 g(\mathbf{y}) = \mathbf{M}^\top \nabla^2 f(\mathbf{My}) \mathbf{M}$. 

Hence the gradient descent update to solve $\min_{\mathbf{y} \in {\mathbb{R}}^n} g(\mathbf{y})$ becomes: 
\begin{align}
{\mathbf{y}}^{k+1} &= {\mathbf{y}}^{k} - \eta \nabla g(\mathbf{y}^{k}) \\
&= {\mathbf{y}}^{k} - \eta {\mathbf{M}}^\top \nabla f(\mathbf{M}\mathbf{y}^{k}) \\
\end{align}


Pre-multiplying by $\mathbf{M}$, we have:
\begin{align}
{\mathbf{M}\mathbf{y}}^{k+1} &= {\mathbf{M}\mathbf{y}}^{k} -  \eta \mathbf{MM}^\top \nabla f(\mathbf{M}\mathbf{y}^{k})  \\
\implies \mathbf{x}^{k+1} &= \mathbf{x}^{k} - \eta \mathbf{MM}^\top \nabla f({\mathbf{x}}^{k}) 
\end{align}

Letting $\mathbf{D} = \mathbf{MM}^\top$, we see that the update is of the form:
\begin{align}
\mathbf{x}^{k+1} &= \mathbf{x}^{k} - \eta \mathbf{D} \nabla f({\mathbf{x}}^{k}) 
\end{align}

Note that the matrix $\mathbf{D}$ is symmetric and positive definite and hence can be written as $\mathbf{D} = \mathbf{B}^2$, where $\mathbf{B}$ is also symmetric and positive definite. 

Denoting $\mathbf{B}= \mathbf{D}^{\frac{1}{2}}$, we see that a useful choice for the matrix $\mathbf{M}$ is $\mathbf{M} = \mathbf{B} = \mathbf{D}^{\frac{1}{2}}$. 

The matrix $\mathbf{D}$ is called a $\textbf{scaling}$ matrix and helps in scaling the Hessian. 

We will consider $\mathbf{D}$ to be a diagonal matrix. Thus it would be useful to find a useful candidate of the scaling matrix at each iteration which could help in significant progress of the iterates towards the optimal solution. 




This discussion leads to the following algorithm:
\begin{align}
& \textbf{Input:} \text{ Starting point $x^0$, Stopping tolerance $\tau$}  \\
& \textbf{Initialize } k=0 \\ 
& \mathbf{p}^k =-\nabla f(\mathbf{x}^k) \\ 
&\textbf{While } \| \mathbf{p}^k \|_2 > \tau \text{ do:}  \\   
&\quad \quad \text{ Choose a suitable scaling matrix }\mathbf{D}^k. \\ 
&\quad \quad \eta^k = \arg\min_{\eta\geq 0} f(\mathbf{x}^k + \eta  \mathbf{D}^k \mathbf{p}^k) = \arg\min_{\eta\geq 0} f(\mathbf{x}^k - \eta  \mathbf{D}^k \nabla f(\mathbf{x}^k)) \\
&\quad \quad \mathbf{x}^{k+1} = \mathbf{x}^k + \eta^k \mathbf{D}^k \mathbf{p}^k = \mathbf{x}^k - \eta^k \mathbf{D}^k \nabla f(\mathbf{x}^k)  \\ 
&\quad \quad k = {k+1} \\ 
&\textbf{End While} \\
&\textbf{Output: } \mathbf{x}^k
\end{align}

In [ ]:
def compute_D_k(x):
  assert type(x) is np.ndarray
  assert len(x) == 2
  #hess = evalh(x)
  #print(hess[0][1])
  return np.array([[np.sqrt(1./1500.), 0.],[0., 1.]]), np.array([[1./1500., 0.],[0., 1.]])
  
  #np.array([[np.sqrt(1./(20000.+2.*(x[1]**2))), 0.],[0., np.sqrt(1./(2.*(x[0]**2)))]]), np.array([[1./(20000.+2.*(x[1]**2)), 0.],[0., 1./(2.*(x[0]**2))]])
  #return np.array([[np.sqrt(1./(-4000.*(x[1]-3.*x[0]**2)+2.)), 0.],[0., np.sqrt(1./2000)]]), np.array([[1./(-4000.*(x[1]-3.*x[0]**2)+2.), 0.],[0., 1./2000]])
  #hess_inv = np.linalg.inv(evalh(x))
  #hess_inv_sqrt = sqrtm(hess_inv)
  #return hess_inv_sqrt, hess_inv
  #return np.array([[np.sqrt(1./32002), 0], [0, np.sqrt(1./32002)]]), np.array([[1./32002, 0], [0, np.sqrt(1./32002)]])

In [ ]:
x = np.array([5., 5.])
D_k_sqrt,D_k = compute_D_k(x)
print(D_k)
hess=evalh(x)
find_condition_number_matrix(hess)
print(hess)
DHessD=np.matmul(D_k_sqrt, np.matmul( hess, D_k_sqrt))
print(DHessD)
find_condition_number_matrix(DHessD)


[[6.66666667e-04 0.00000000e+00]
 [0.00000000e+00 1.00000000e+00]]
[[38.72980074  0.05034165]
 [ 0.05034165  0.99873206]]
1504.016046343423
[[1.5e+03 2.0e+00]
 [2.0e+00 1.0e+00]]
[[1.         0.05163978]
 [0.05163978 1.        ]]
1.1089032980269364


1.1089032980269364

In [ ]:
#Now we will define a function which will compute and return the function value 
def evalf(x):  
  #Input: x is a numpy array of size 2 
  assert type(x) is np.ndarray and len(x) == 2 #do not allow arbitrary arguments 
  #after checking if the argument is valid, we can compute the objective function value
  #compute the function value and return it 
  return (1500.0*(x[0]**2)+ 4.0*x[0]*x[1] + (x[1]**2))


In [ ]:
my_x = np.array([-0.5921619, 110.92350273])
evalf(my_x)

12567.268343084725

In [ ]:
#Now we will define a function which will compute and return the gradient value as a numpy array 
def evalg(x):  
  #Input: x is a numpy array of size 2 
  assert type(x) is np.ndarray and len(x) == 2 #do not allow arbitrary arguments 
  #after checking if the argument is valid, we can compute the gradient value
  #compute the gradient value and return it 
  return np.array([3000.0*x[0]+4.0*x[1], 4.0*x[0]+2.0*x[1]])

In [ ]:
#Complete the module to compute the steplength by using the closed-form expression
def compute_steplength_exact(gradf, A): #add appropriate arguments to the function 
  assert type(gradf) is np.ndarray and len(gradf) == 2 
  assert type(A) is np.ndarray and A.shape[0] == 2 and  A.shape[1] == 2 #allow only a 2x2 array
   
  #Complete the code 
  step_length = 0.
  return step_length

In [ ]:
#Complete the module to compute the steplength by using simple backtracking
def compute_steplength_backtracking(x, gradf, alpha_start, rho, gamma): #add appropriate arguments to the function 
  assert type(x) is np.ndarray and len(gradf) == 2 
  assert type(gradf) is np.ndarray and len(gradf) == 2 
  assert type(alpha_start) is float and alpha_start>=0. 
  assert type(rho) is float and rho>=0.
  assert type(gamma) is float and gamma>=0. 
  
  
  #Complete the code
  alpha = 0
  return alpha

In [ ]:
def compute_steplength_backtracking_dirn(x, gradf, direction, alpha_start, rho, gamma): #add appropriate arguments to the function 
  assert type(x) is np.ndarray and len(gradf) == 2 
  assert type(gradf) is np.ndarray and len(gradf) == 2 
  assert type(alpha_start) is float and alpha_start>=0. 
  assert type(rho) is float and rho>=0.
  assert type(gamma) is float and gamma>=0. 
  
  
  
  alpha = alpha_start
  gradf_dot_direction = np.dot(gradf, direction)
  evalf_x = evalf(x)
  while evalf(np.add(x,np.multiply(-alpha, direction)))>evalf_x  - alpha * gamma * gradf_dot_direction:
    alpha = rho * alpha
  #print('final step length:',alpha)
  return alpha

In [ ]:
#line search type 
CONSTANT_STEP_LENGTH = 3
BACKTRACKING_LINE_SEARCH = 2
EXACT_LINE_SEARCH = 1

In [ ]:
def find_minimizer_gd(start_x, tol, line_search_type, *args):
  #Input: start_x is a numpy array of size 2, tol denotes the tolerance and is a positive float value
  assert type(start_x) is np.ndarray and len(start_x) == 2 #do not allow arbitrary arguments 
  assert type(tol) is float and tol>=0 
  A = np.array([[1500., 2.],[2.,1.]])
  x = start_x
  g_x = evalg(x)

  if line_search_type == BACKTRACKING_LINE_SEARCH:
    if args is None:
      err_msg = 'Line search type: BACKTRACKING_LINE_SEARCH, but did not receive any args. Please check!'
      raise ValueError(err_msg)
    elif len(args)<3 :
      err_msg = 'Line search type: BACKTRACKING_LINE_SEARCH, but did not receive three args. Please check!'
      raise ValueError(err_msg)
    else:
      alpha_start = float(args[0])
      rho = float(args[1])
      gamma = float(args[2])
  k = 0
  #print('iter:',k, ' x:', x, ' f(x):', evalf(x), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))

  while (np.linalg.norm(g_x) > tol): #continue as long as the norm of gradient is not close to zero upto a tolerance tol
    #step_length = compute_steplength_exact(g_x, A) #call the new function you wrote to compute the steplength
    if line_search_type == BACKTRACKING_LINE_SEARCH:
      step_length = compute_steplength_backtracking(x,g_x, alpha_start,rho, gamma) #call the new function you wrote to compute the steplength
    elif line_search_type == EXACT_LINE_SEARCH:
      raise ValueError('Exact search not yet implemented')
      #step_length= compute_steplength_exact(g_x, A)
    elif line_search_type == CONSTANT_STEP_LENGTH:  
      step_length = 0.1
    else:
      err_msg = 'line search type:'+str(line_search_type)+' unknown! please check ...'
      raise ValueError(err_msg)

    x = np.subtract(x, np.multiply(step_length,g_x)) #update x = x - step_length*g_x
    k += 1 #increment iteration
    g_x = evalg(x) #compute gradient at new point
    if k%100 == 0:
      print('iter:',k, ' x:', x, ' f(x):', evalf(x), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))
  
  print('alpha_start:',alpha_start, 'rho:', rho, 'iter:',k, ' x:', x, ' f(x):', evalf(x), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))      
  return x, evalf(x),k 


In [ ]:
def find_minimizer_gdscaling(start_x, tol, line_search_type, *args):
  #Input: start_x is a numpy array of size 2, tol denotes the tolerance and is a positive float value
  assert type(start_x) is np.ndarray and len(start_x) == 2 #do not allow arbitrary arguments 
  assert type(tol) is float and tol>=0 

  x = start_x
  g_x = evalg(x)

  if line_search_type == BACKTRACKING_LINE_SEARCH:
    if args is None:
      err_msg = 'Line search type: BACKTRACKING_LINE_SEARCH, but did not receive any args. Please check!'
      raise ValueError(err_msg)
    elif len(args)<3 :
      err_msg = 'Line search type: BACKTRACKING_LINE_SEARCH, but did not receive three args. Please check!'
      raise ValueError(err_msg)
    else:
      alpha_start = float(args[0])
      rho = float(args[1])
      gamma = float(args[2])
  k = 0
  #print('iter:',k, ' x:', x, ' f(x):', evalf(x), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))

  while (np.linalg.norm(g_x) > tol): #continue as long as the norm of gradient is not close to zero upto a tolerance tol
    D_k_sqrt,D_k = compute_D_k(x)
    

    #step_length = compute_steplength_exact(g_x, A) #call the new function you wrote to compute the steplength
    if line_search_type == BACKTRACKING_LINE_SEARCH:
      #step_length = compute_steplength_backtracking_dirn() #call the new function with appropriate arguments to compute the steplength 
    elif line_search_type == EXACT_LINE_SEARCH:
      raise ValueError('Exact search not yet implemented')
    elif line_search_type == CONSTANT_STEP_LENGTH:  
      step_length = 0.1
    else:
      err_msg = 'line search type:'+str(line_search_type)+' unknown! please check ...'
      raise ValueError(err_msg)
    #Scaled Hessian 
    scaled_Hessian = np.matmul(D_k_sqrt, np.matmul(evalh(x),D_k_sqrt))
    
    #print('condition number of scaled Hessian:',find_condition_number_matrix(scaled_Hessian) )
    x = np.subtract(x, np.multiply(step_length,D_k_mul_g_x)) #update x = x - step_length*g_x
    k += 1 #increment iteration
    g_x = evalg(x) #compute gradient at new point
    #if 0: # k%10000 == 0:
    print('iter:',k, ' x:', x, ' f(x):', evalf(x), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x), 'Hessian cond num:', find_condition_number_matrix(scaled_Hessian))
  print('alpha_start:',alpha_start, 'rho:', rho, 'iter:',k, ' x:', x, ' f(x):', evalf(x), ' grad at x:', g_x, ' gradient norm:', np.linalg.norm(g_x))      
  return x, evalf(x),k 


In [ ]:
my_start_x = np.array([1.,4000.])
my_tol= 1e-12
alpha_start = 1.
rho = 0.5
gamma = 0.5
#x_opt, opt_fval, num_iters = find_minimizer_gd(my_start_x, my_tol, 1)
x_opt, opt_fval, num_iters = find_minimizer_gd(my_start_x, my_tol, 2,alpha_start, rho, gamma)

iter: 100  x: [-2.54036972e+00  3.39229410e+03]  f(x): 11482868.773155304  grad at x: [5948.06725698 6774.42672692]  gradient norm: 9015.118500166325
iter: 200  x: [1.22341526e-01 2.86570279e+03]  f(x): 8213677.304012541  grad at x: [11829.83573425  5731.89494416]  gradient norm: 13145.327426513373
iter: 300  x: [  -5.03012151 2426.7621057 ]  f(x): 5878299.868176542  grad at x: [-5383.31610584  4833.40372535]  gradient norm: 7234.7690956696015
iter: 400  x: [  -4.15485624 2054.0587983 ]  f(x): 4210914.516376751  grad at x: [-4248.33351619  4091.49817166]  gradient norm: 5898.194211237012
iter: 500  x: [  -3.43568202 1738.59543872]  f(x): 3016526.9216023567  grad at x: [-3352.66430783  3463.44814936]  gradient norm: 4820.35590431472
iter: 600  x: [  -2.84405663 1471.58109072]  f(x): 2160942.8538846574  grad at x: [-2645.84551263  2931.78595493]  gradient norm: 3949.1603363544345
iter: 700  x: [  -3.40856946 1243.14251685]  f(x): 1545881.4853983072  grad at x: [-5253.13832493  2472.65075

In [ ]:
my_start_x = np.array([1.,4000.])
my_tol= 1e-12
alpha_start = 1.
rho = 0.5
gamma = 0.5
x_opt, opt_fval, num_iters = find_minimizer_gdscaling(my_start_x, my_tol, 2, alpha_start, rho, gamma)

[[38.72980074  0.05034165]
 [ 0.05034165  0.99873206]]
1.1089032980269364
iter: 1  x: [  -2.16666667 1999.        ]  f(x): 3985718.0  grad at x: [1496.         3989.33333333]  gradient norm: 4260.609867665009 Hessian cond num: 1.1089032980269364
[[38.72980074  0.05034165]
 [ 0.05034165  0.99873206]]
1.1089032980269364
iter: 2  x: [  -2.416      1001.66666667]  f(x): 1002411.5884444444  grad at x: [-3241.33333333  1993.66933333]  gradient norm: 3805.3855505653505 Hessian cond num: 1.1089032980269364
[[38.72980074  0.05034165]
 [ 0.05034165  0.99873206]]
1.1089032980269364
iter: 3  x: [-1.33555556  4.832     ]  f(x): 2673.097569185184  grad at x: [-3987.33866667     4.32177778]  gradient norm: 3987.3410088000082 Hessian cond num: 1.1089032980269364
[[38.72980074  0.05034165]
 [ 0.05034165  0.99873206]]
1.1089032980269364
iter: 4  x: [-0.00644267  2.67111111]  f(x): 7.1282601844938265  grad at x: [-8.64355556  5.31645156]  gradient norm: 10.147694801507992 Hessian cond num: 1.108903298026

$\Large{\text{References:}}$



1.   Jorge Nocedal and Stephen J. Wright, $\textit{Numerical Optimization}$. Springer series in Operations Research,  Second Edition, 2006.
2.   Amir Beck, $\textit{Introduction to Nonlinear Optimization}$. MOS-SIAM series on Optimization, 2014.
3. Dimitri P. Bertsekas. $\textit{Nonlinear Programming}$. Athena Scientific, Second Edition, 1999.
